In [1]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import hpelm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from skimage.feature import hog
from skimage import color, io

In [2]:
dataset_dir = 'The IQ-OTHNCCD lung cancer dataset/'
X = []
y = []

for label, class_name in enumerate(os.listdir(dataset_dir)):
    class_folder = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_folder):
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_gray = color.rgb2gray(img_array)
            features, _ = hog(img_gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
            X.append(features)
            y.append(label)

X = np.array(X)
y = np.array(y)

In [3]:
y

array([0, 0, 0, ..., 2, 2, 2])

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

In [5]:
elm = hpelm.HPELM(X_train.shape[1], y_train_one_hot.shape[1])
elm.add_neurons(250, "sigm")

elm.train(X_train, y_train_one_hot, "c")

In [6]:
# Memprediksi data uji
y_pred_prob = elm.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

In [7]:
# Menyiapkan label uji dalam format yang sesuai
y_test_labels = np.argmax(y_test_one_hot, axis=1)
accuracy = accuracy_score(y_test_labels, y_pred)
classification_rep = classification_report(y_test_labels, y_pred)
confusion_mat = confusion_matrix(y_test_labels, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Classification Report:\n{classification_rep}")
print(f"Confusion Matrix:\n{confusion_mat}")

Accuracy: 0.91
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.62      0.75        29
           1       0.95      0.97      0.96       109
           2       0.87      0.94      0.90        82

    accuracy                           0.91       220
   macro avg       0.92      0.84      0.87       220
weighted avg       0.92      0.91      0.91       220

Confusion Matrix:
[[ 18   2   9]
 [  0 106   3]
 [  1   4  77]]


In [8]:
elm.save('model_elm')